In [1]:
from __future__ import division, print_function, absolute_import

import os
from timeit import time
import warnings
import sys
import cv2
import numpy as np
from PIL import Image
from yolo import YOLO

from deep_sort import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet
from deep_sort.detection import Detection as ddet
warnings.filterwarnings('ignore')
import imageio

Using TensorFlow backend.


In [2]:
def main(yolo):

   # Definition of the parameters
    max_cosine_distance = 0.3
    nn_budget = None
    nms_max_overlap = 1.0
    
   # deep_sort 
    model_filename = 'model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename,batch_size=1)
    
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    writeVideo_flag = True 
    
    video_capture = cv2.VideoCapture('cartest.mp4')

    if writeVideo_flag:
    # Define the codec and create VideoWriter object
        w = int(video_capture.get(3))
        h = int(video_capture.get(4))
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter('output.avi', fourcc, 15, (w, h))
        list_file = open('detection.txt', 'w')
        frame_index = -1 
        
    fps = 0.0
    while True:
        ret, frame = video_capture.read()  # frame shape 640*480*3
        if ret != True:
            break
        t1 = time.time()

       # image = Image.fromarray(frame)
        image = Image.fromarray(frame[...,::-1]) #bgr to rgb
        boxs = yolo.detect_image(image)
       # print("box_num",len(boxs))
        features = encoder(frame,boxs)
        
        # score to 1.0 here).
        detections = [Detection(bbox, 1.0, feature) for bbox, feature in zip(boxs, features)]
        
        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]
        
        # Call the tracker
        tracker.predict()
        tracker.update(detections)
        
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue 
            bbox = track.to_tlbr()
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
            cv2.putText(frame, str(track.track_id),(int(bbox[0]), int(bbox[1])),0, 5e-3 * 200, (0,255,0),2)

        for det in detections:
            bbox = det.to_tlbr()
            cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,0,0), 2)
            
        cv2.imshow('', frame)
        
        if writeVideo_flag:
            # save a frame
            out.write(frame)
            frame_index = frame_index + 1
            list_file.write(str(frame_index)+' ')
            if len(boxs) != 0:
                for i in range(0,len(boxs)):
                    list_file.write(str(boxs[i][0]) + ' '+str(boxs[i][1]) + ' '+str(boxs[i][2]) + ' '+str(boxs[i][3]) + ' ')
            list_file.write('\n')
            
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        print("fps= %f"%(fps))
        
        # Press Q to stop!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    if writeVideo_flag:
        out.release()
        list_file.close()
    cv2.destroyAllWindows()

In [3]:
main(YOLO())

model_data/yolo.h5 model, anchors, and classes loaded.
fps= 0.063436
fps= 0.830535
fps= 4.128507
fps= 4.246557
fps= 7.345374
fps= 8.961901
fps= 9.492869
fps= 9.982076
fps= 10.171033
fps= 8.530813
fps= 9.403558
fps= 9.991646
fps= 9.909971
fps= 9.932124
fps= 10.081778
fps= 10.428106
fps= 10.342392
fps= 10.564009
fps= 10.515830
fps= 10.619638
fps= 10.528939
fps= 10.441779
fps= 10.444520
fps= 10.387367
fps= 10.268424
fps= 10.325260
fps= 10.176309
fps= 10.420783
fps= 10.390130
fps= 10.256175
fps= 10.403225
fps= 10.381492
fps= 10.302771
fps= 10.025094
fps= 9.742265
fps= 9.786570
fps= 10.170493
fps= 10.253676
fps= 10.349103
fps= 10.303643
fps= 10.166988
fps= 9.961544
fps= 10.273163
fps= 10.148392
fps= 9.848773
fps= 10.092803
fps= 10.024059
fps= 10.178383
fps= 10.418621
fps= 8.424096
fps= 9.431259
fps= 9.693228
fps= 9.910670
fps= 10.068020
fps= 9.818577
fps= 9.886320
fps= 10.056157
fps= 10.038225
fps= 10.134853
fps= 10.179763
fps= 10.117323
fps= 10.070268
fps= 9.996083
fps= 8.085680
fps= 8.927

fps= 8.414421
fps= 8.350371
fps= 8.572874
fps= 8.571350
fps= 8.802216
fps= 8.877362
fps= 8.723620
fps= 8.919370
fps= 8.898835
fps= 8.698010
fps= 8.670873
fps= 8.513222
fps= 8.693225
fps= 8.863066
fps= 8.644475
fps= 8.644088
fps= 8.499846
fps= 8.571960
fps= 8.722604
fps= 8.720633
fps= 8.917896
fps= 9.016518
fps= 8.984484
fps= 9.008774
fps= 8.753005
fps= 8.852700
fps= 8.902585
fps= 8.848972
fps= 8.861072
fps= 9.160125
fps= 9.096536
fps= 8.984901
fps= 8.814328
fps= 8.964743
fps= 8.918986
fps= 8.818896
fps= 9.008809
fps= 8.980620
fps= 8.926924
fps= 9.148788
fps= 9.176389
fps= 9.318499
fps= 9.344722
fps= 9.148577
fps= 9.090830
fps= 9.230762
fps= 9.089382
fps= 9.104817
fps= 9.109998
fps= 9.284663
fps= 9.158844
fps= 9.055638
fps= 9.004054
fps= 9.143952
fps= 9.048163
fps= 9.040613
fps= 9.162252
fps= 9.138745
fps= 8.678688
fps= 8.552252
fps= 8.918123
fps= 8.818492
fps= 8.966825
fps= 9.041052
fps= 9.120421
fps= 8.996806
fps= 9.014895
fps= 9.237037
fps= 8.940387
fps= 9.112211
fps= 9.155446
fps= 9

fps= 8.665578
fps= 9.018210
fps= 8.830946
fps= 8.700395
fps= 8.709655
fps= 8.871427
fps= 9.035074
fps= 8.993753
fps= 9.013389
fps= 8.943271
fps= 8.720228
fps= 8.836349
fps= 8.595968
fps= 8.774152
fps= 8.903637
fps= 8.928034
fps= 8.823466
fps= 8.967055
fps= 8.962009
fps= 9.080385
fps= 9.054556
fps= 8.888703
fps= 8.841975
fps= 8.897175
fps= 8.697223
fps= 8.708087
fps= 8.713465
fps= 9.042101
fps= 8.957655
fps= 9.036447
fps= 8.915921
fps= 8.974472
fps= 8.772114
fps= 9.160678
fps= 9.056584
fps= 9.086020
fps= 9.186394
fps= 8.952664
fps= 8.798191
fps= 8.956685
fps= 8.763308
fps= 8.939204
fps= 8.829087
fps= 8.851139
fps= 8.638452
fps= 8.716905
fps= 8.680267
fps= 8.737868
fps= 8.728365
fps= 8.723694
fps= 8.721269
fps= 8.836831
fps= 8.777810
fps= 8.532180
fps= 8.309105
fps= 8.403179
fps= 8.244565
fps= 7.949300
fps= 7.922163
fps= 8.104324
fps= 8.095177
fps= 8.156903
fps= 8.057256
fps= 8.241570
fps= 8.298587
fps= 8.292577
fps= 8.189357
fps= 8.203986
fps= 8.279811
fps= 8.387302
fps= 8.174192
fps= 8

fps= 8.648141
fps= 8.760666
fps= 8.628934
fps= 8.492202
fps= 8.531031
fps= 8.514082
fps= 8.616463
fps= 8.708079
fps= 8.713524
fps= 8.832959
fps= 8.853065
fps= 8.748374
fps= 8.890738
fps= 9.044709
fps= 9.038906
fps= 9.077042
fps= 8.198070
fps= 8.208415
fps= 8.463647
fps= 8.748355
fps= 8.587094
fps= 8.542140
fps= 8.157405
fps= 8.554919
fps= 8.386782
fps= 8.406291
fps= 8.415994
fps= 8.456598
fps= 8.625960
fps= 8.789177
fps= 8.754037
fps= 8.486382
fps= 8.679843
fps= 8.137906
fps= 8.428383
fps= 8.771751
fps= 8.985256
fps= 8.929195
fps= 7.970175
fps= 7.783092
fps= 8.000837
fps= 8.285331
fps= 8.464516
fps= 8.480833
fps= 8.638133
fps= 8.643020
fps= 8.643360
fps= 8.500697
fps= 8.535270
fps= 8.784643
fps= 8.535597
fps= 8.377089
fps= 8.664713
fps= 8.808554
fps= 8.880483
fps= 8.583673
fps= 8.302523
fps= 8.399820
fps= 8.309209
fps= 8.476482
fps= 8.416034
